<a href="https://colab.research.google.com/github/himoyuzuki/colab_analytics/blob/main/%E6%A0%AA%E4%BE%A1%E6%83%85%E5%A0%B1%E3%82%92%E5%8F%96%E5%BE%97%E3%81%99%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 株価情報を取得する

## 用語解説

- 日経平均...東京プライム市場の中から市場流動性の高さや業種のバランスなどを考慮して選んだ225銘柄の平均株価指数のこと
- TOPIX...「Tokyo Stock Price Index」の略。別名東証株価指数。東京プライム市場の全銘柄（約2,000社）が対象
```
TOPIX（ポイント）＝算出時の時価総額※÷基準時の時価総額（1968年1月4日時点）×100
※時価総額＝株価×発行株式数
```


## 分析

In [ ]:
from datetime import date
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import pandas_datareader.data as web
import yfinance as yf

import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests

### 自動車業界主要企業

以下を参考に6つ選出。

[自動車業界 : 企業一覧 : 日経会社情報DIGITAL : 日経電子版](https://www.nikkei.com/nkd/industry/complist/?n_m_code=041)

In [ ]:
# yfinanceから取得
car_df = yf.download(('7203.T', '7267.T', '7201.T', '7269.T', '7261.T', '7270.T'), start='2020-01-01', end='2025-06-30')
car_df = car_df.rename(columns={
    '7203.T': 'トヨタ',
    '7267.T': 'ホンダ',
    '7201.T': '日産',
    '7269.T': 'スズキ',
    '7261.T': 'マツダ',
    '7270.T': 'スバル'
    })

/tmp/ipython-input-42-3117614860.py:2: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  6 of 6 completed


In [ ]:
# グラフ描画用DataFrameの作成
graph_df = car_df[['Close']]
graph_df.columns = graph_df.columns.droplevel(level=0)  # マルチインデックスの1つは不要

# Plotlyグラフの作成
fig = go.Figure()

# 左側のY軸に日経平均を追加
for col in graph_df.columns:
    fig.add_trace(go.Scatter(x=graph_df.index, y=graph_df[col],
                             mode='lines', name=col,
                             yaxis='y1'))

# レイアウト設定（2つのY軸を定義）
fig.update_layout(
    title='自動車業界主要企業の株価推移',
    xaxis_title='Date',
    yaxis=dict(
        title='株価',
        showgrid=True,
        zeroline=True,
    ),
    hovermode='x',
    xaxis_rangeslider_visible=True,
    template='plotly_dark',
)

# グラフを表示
fig.show()

### 日経平均・TOPIX

In [ ]:
# 日経平均株価をyfainanceから取得
nikkei_df = yf.download('^N225', start='2020-01-01', end='2025-06-30')
nikkei_df.columns = nikkei_df.columns.droplevel(level=1)  # マルチインデックスの1つは不要

# TOPIXをpandas_datareaderを使い、stooqから取得（yfinanceで2024年まで取得できなかった）
topix_df = web.DataReader('^TPX', 'stooq', date(2020, 1, 1), date(2025, 6, 30))
topix_df = topix_df.sort_values("Date").reset_index().set_index('Date')

/tmp/ipython-input-29-693933674.py:2: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [ ]:
# グラフ描画用DataFrameの作成
nikkei_df.rename(columns={'Close': 'Nikkei225'}, inplace=True)
topix_df.rename(columns={'Close': 'TOPIX'}, inplace=True)
graph_df = pd.concat([nikkei_df['Nikkei225'], topix_df['TOPIX']], axis=1)

# Plotlyグラフの作成
fig = go.Figure()

# 左側のY軸に日経平均を追加
fig.add_trace(go.Scatter(x=graph_df.index, y=graph_df['Nikkei225'],
                         mode='lines', name='Nikkei 225',
                         yaxis='y1'))

# 右側のY軸にTOPIXを追加
fig.add_trace(go.Scatter(x=graph_df.index, y=graph_df['TOPIX'],
                         mode='lines', name='TOPIX',
                         yaxis='y2'))

# レイアウト設定（2つのY軸を定義）
fig.update_layout(
    title='日経平均とTOPIXの推移',
    xaxis_title='Date',
    yaxis=dict(
        title='Nikkei 225',
        showgrid=True,
        zeroline=True,
    ),
    yaxis2=dict(
        title='TOPIX',
        overlaying='y',
        side='right',
        showgrid=False,
    ),
    hovermode='x',
    xaxis_rangeslider_visible=True,
    template='plotly_dark',
)

# グラフを表示
fig.show()

### 企業分析

In [ ]:
STOCK = yf.Ticker("7203.T")

In [ ]:
# STOCK.info から主要な情報を抽出して表示

info = STOCK.info

print("--- 企業概要 ---")
print(f"企業名: {info.get('longName', 'N/A')}")
print(f"業種: {info.get('industryDisp', 'N/A')}")
print(f"セクター: {info.get('sectorDisp', 'N/A')}")
print(f"所在地: {info.get('city', 'N/A')}, {info.get('country', 'N/A')}")
print(f"従業員数: {info.get('fullTimeEmployees', 'N/A')}")
print(f"ウェブサイト: {info.get('website', 'N/A')}")
print(f"事業内容の概要:\n{info.get('longBusinessSummary', 'N/A')}\n")

print("--- 株価概要 ---")
print(f"現在の株価: {info.get('currentPrice', 'N/A')} {info.get('currency', 'N/A')}")
print(f"前日終値: {info.get('previousClose', 'N/A')} {info.get('currency', 'N/A')}")
print(f"今日の高値: {info.get('dayHigh', 'N/A')} {info.get('currency', 'N/A')}")
print(f"今日の安値: {info.get('dayLow', 'N/A')} {info.get('currency', 'N/A')}")
print(f"52週高値: {info.get('fiftyTwoWeekHigh', 'N/A')} {info.get('currency', 'N/A')}")
print(f"52週安値: {info.get('fiftyTwoWeekLow', 'N/A')} {info.get('currency', 'N/A')}")
print(f"時価総額: {info.get('marketCap', 'N/A')} {info.get('financialCurrency', 'N/A')}")
print(f"平均取引量 (3ヶ月): {info.get('averageDailyVolume3Month', 'N/A')}\n")

print("--- 財務情報 (一部) ---")
print(f"株価収益率 (PER, Trailing): {info.get('trailingPE', 'N/A')}")
print(f"株価収益率 (PER, Forward): {info.get('forwardPE', 'N/A')}")
print(f"配当利回り: {info.get('dividendYield', 'N/A')}")
print(f"発行済株式数: {info.get('sharesOutstanding', 'N/A')}")
print(f"直近会計年度末: {info.get('lastFiscalYearEnd', 'N/A')}") # Unixタイムスタンプの場合があります
print(f"次期会計年度末: {info.get('nextFiscalYearEnd', 'N/A')}") # Unixタイムスタンプの場合があります
print(f"直近四半期末: {info.get('mostRecentQuarter', 'N/A')}") # Unixタイムスタンプの場合があります
print(f"純利益 (Trailing 12M): {info.get('netIncomeToCommon', 'N/A')} {info.get('financialCurrency', 'N/A')}")
print(f"自己資本利益率 (ROE): {info.get('returnOnEquity', 'N/A')}\n")

print("--- アナリスト評価 ---")
print(f"推奨平均: {info.get('averageAnalystRating', 'N/A')}")
print(f"推奨キー: {info.get('recommendationKey', 'N/A')}")
print(f"アナリスト数: {info.get('numberOfAnalystOpinions', 'N/A')}")
print(f"目標株価 (平均): {info.get('targetMeanPrice', 'N/A')} {info.get('currency', 'N/A')}")
print(f"目標株価 (高): {info.get('targetHighPrice', 'N/A')} {info.get('currency', 'N/A')}")
print(f"目標株価 (低): {info.get('targetLowPrice', 'N/A')} {info.get('currency', 'N/A')}")

--- 企業概要 ---
企業名: Toyota Motor Corporation
業種: Auto Manufacturers
セクター: Consumer Cyclical
所在地: Toyota, Japan
従業員数: 383853
ウェブサイト: https://global.toyota/en
事業内容の概要:
Toyota Motor Corporation designs, manufactures, assembles, and sells passenger vehicles, minivans and commercial vehicles, and related parts and accessories in Japan, North America, Europe, Asia, Central and South America, Oceania, Africa, and the Middle East. It operates through Automotive, Financial Services, and All Other segments. The company offers subcompact and compact cars; mini-vehicles; mid-size, luxury, sports, and specialty cars; recreational and sport-utility vehicles; pickup trucks; minivans; trucks; and buses. It offers vehicles under the Toyota and Lexus brand names. Further, the company provides financial services, such as retail financing and leasing, wholesale financing, insurance, and credit cards. Additionally, it operates GAZOO.com, a web portal for automobile information, as well as engages in telecomm

In [ ]:
# STOCK オブジェクトから財務三表を取得

# 貸借対照表 (Balance Sheet)
print("--- 貸借対照表 ---")
balance_sheet = STOCK.balance_sheet
if balance_sheet is not None and not balance_sheet.empty:
    display(balance_sheet)
else:
    print("貸借対照表のデータが見つかりませんでした。")

--- 貸借対照表 ---


,2025-03-31,2024-03-31,2023-03-31,2022-03-31
Treasury Shares Number,2.746058e+09,2.840815e+09,2.749808e+09,2.536686e+09
Ordinary Shares Number,1.304893e+10,1.347417e+10,1.356518e+10,1.377830e+10
Share Issued,1.579499e+10,1.631499e+10,1.631499e+10,1.631499e+10
Net Debt,2.927712e+13,2.668715e+13,2.140719e+13,1.996177e+13
Total Debt,3.879288e+13,3.656178e+13,2.938027e+13,2.649636e+13
...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,1.591816e+13,1.411423e+13,9.232641e+12,8.620903e+12
Other Short Term Investments,6.935759e+12,4.702168e+12,1.715675e+12,2.507248e+12
Cash And Cash Equivalents,8.982404e+12,9.412060e+12,7.516966e+12,6.113655e+12
Cash Equivalents,2.637713e+12,3.166803e+12,1.568669e+12,1.482773e+12


In [ ]:
# 損益計算書 (Income Statement)
print("\n--- 損益計算書 ---")
income_statement = STOCK.financials
if income_statement is not None and not income_statement.empty:
    display(income_statement)
else:
    print("損益計算書のデータが見つかりませんでした。")


--- 損益計算書 ---


,2025-03-31,2024-03-31,2023-03-31,2022-03-31,2021-03-31
Tax Effect Of Unusual Items,2.089089e+11,8.561091e+10,7.482368e+10,8.708560e+10,NaN
Tax Rate For Calcs,2.530000e-01,2.720000e-01,3.200000e-01,2.800000e-01,NaN
Normalized EBITDA,7.924202e+12,8.802138e+12,5.522169e+12,5.533850e+12,NaN
Total Unusual Items,8.257270e+11,3.147460e+11,2.338240e+11,3.110200e+11,NaN
Total Unusual Items Excluding Goodwill,8.257270e+11,3.147460e+11,2.338240e+11,3.110200e+11,NaN
Net Income From Continuing Operation Net Minority Interest,4.765086e+12,4.944933e+12,2.451318e+12,2.850110e+12,NaN
Reconciled Depreciation,2.251233e+12,2.087066e+12,2.039904e+12,1.821880e+12,NaN
Reconciled Cost Of Revenue,3.845867e+13,3.572701e+13,3.084128e+13,2.540783e+13,NaN
EBITDA,8.749929e+12,9.116884e+12,5.755993e+12,5.844870e+12,NaN
EBIT,6.498696e+12,7.029818e+12,3.716089e+12,4.022990e+12,NaN


In [ ]:
# キャッシュフロー計算書 (Cash Flow Statement)
print("\n--- キャッシュフロー計算書 ---")
cashflow_statement = STOCK.cashflow
if cashflow_statement is not None and not cashflow_statement.empty:
    display(cashflow_statement)
else:
    print("キャッシュフロー計算書のデータが見つかりませんでした。")


--- キャッシュフロー計算書 ---


,2025-03-31,2024-03-31,2023-03-31,2022-03-31
Free Cash Flow,-1.560993e+12,-8.420210e+11,-7.507560e+11,-1.076290e+11
Repayment Of Debt,-1.087226e+13,-8.752329e+12,-8.353033e+12,-8.843665e+12
Issuance Of Debt,1.338158e+13,1.205735e+13,9.276918e+12,8.122678e+12
Capital Expenditure,-5.257927e+12,-5.048394e+12,-3.705832e+12,-3.830244e+12
End Cash Position,8.982404e+12,9.412060e+12,7.516966e+12,6.113655e+12
Beginning Cash Position,9.412060e+12,7.516966e+12,6.113655e+12,5.100857e+12
Effect Of Exchange Rate Changes,-1.340890e+11,1.899140e+11,1.033050e+11,3.341950e+11
Changes In Cash,-2.955670e+11,1.705180e+12,1.300006e+12,6.786030e+11
Financing Cash Flow,1.972360e+11,2.497558e+12,-5.618000e+10,-2.466516e+12
Cash Flow From Continuing Financing Activities,1.972350e+11,2.497558e+12,-5.618100e+10,-2.466516e+12


In [ ]:
# balance_sheet DataFrameの項目名を日本語に変換して表示

japanese_names = {
    'Treasury Shares Number': '自己株式数',
    'Ordinary Shares Number': '普通株式数',
    'Share Issued': '発行済株式数',
    'Net Debt': '純有利子負債',
    'Total Debt': '有利子負債合計',
    'Total Liabilities Net Minority Interest': '負債合計 (非支配持分控除後)',
    'Total Non Current Liabilities Net Minority Interest': '固定負債合計 (非支配持分控除後)',
    'Other Non Current Liabilities': 'その他の固定負債',
    'Long Term Debt And Capital Lease Obligation': '長期借入金及びリース債務',
    'Long Term Debt': '長期借入金',
    'Current Liabilities': '流動負債',
    'Other Current Liabilities': 'その他の流動負債',
    'Current Deferred Liabilities': '流動繰延負債',
    'Current Debt And Capital Lease Obligation': '短期借入金及びリース債務',
    'Current Debt': '短期借入金',
    'Accounts Payable': '買掛金',
    'Retained Earnings': '利益剰余金',
    'Capital Lease Obligations': 'リース債務',
    'Common Stock': '普通株式',
    'Total Equity Gross Minority Interest': '自己資本合計 (非支配持分含む)',
    'Stockholders Equity': '株主資本',
    'Total Liabilities And Equity': '負債及び自己資本合計',
    'Total Assets': '総資産',
    'Total Non Current Assets': '固定資産合計',
    'Other Non Current Assets': 'その他の固定資産',
    'Investments And Advances': '投資その他の資産',
    'Other Investments': 'その他有価証券',
    'Property Plant And Equipment Net': '有形固定資産 (純額)',
    'Accumulated Depreciation': '減価償却累計額',
    'Property Plant And Equipment Gross': '有形固定資産 (総額)',
    'Current Assets': '流動資産',
    'Other Current Assets': 'その他の流動資産',
    'Inventory': '棚卸資産',
    'Receivables': '売上債権',
    'Accounts Receivable': '売掛金',
    'Cash Cash Equivalents And Short Term Investments': '現金及び現金同等物、短期投資',
    'Other Short Term Investments': 'その他の短期投資',
    'Cash And Cash Equivalents': '現金及び現金同等物',
    'Cash Equivalents': '現金同等物',
    'Cash Financial': '現金（金融事業）',
    'Deferred Tax Liabilities': '繰延税金負債',
    'Deferred Tax Assets': '繰延税金資産',
    'GoodwillAndOtherIntangibleAssets': 'のれん及びその他の無形固定資産',
    'Goodwill': 'のれん',
    'IntangibleAssets': '無形固定資産',
    'Invested Capital': '投下資本',
    'Working Capital': '運転資本',
    'Net Tangible Assets': '純有形固定資産',
    'PayablesAndAccruedExpenses': '買入債務及び未払費用',
    'InterestandOtherReceivable': '受取利息・その他の受取債権',
    'LiabilitiesAndStockholdersEquity': '負債及び株主資本合計',
    'LongTermCapitalLeaseObligation': '長期リース債務',
    'ShortTermCapitalLeaseObligation': '短期リース債務',
    'Net PPE': '有形固定資産 (純額)', # 別名の場合
    'Payable': '買入債務', # 別名の場合
    'Accounts Payable And Accrued Expense': '買掛金及び未払費用', # 別名の場合
}

# カラム名を日本語にリネーム
# エラーを避けるため、存在しないカラムは無視します
balance_sheet_japanese = balance_sheet.rename(index=japanese_names)

# 日本語カラム名でDataFrameを表示
print("--- 貸借対照表 (日本語項目名) ---")
display(balance_sheet_japanese)

--- 貸借対照表 (日本語項目名) ---


,2025-03-31,2024-03-31,2023-03-31,2022-03-31
自己株式数,2.746058e+09,2.840815e+09,2.749808e+09,2.536686e+09
普通株式数,1.304893e+10,1.347417e+10,1.356518e+10,1.377830e+10
発行済株式数,1.579499e+10,1.631499e+10,1.631499e+10,1.631499e+10
純有利子負債,2.927712e+13,2.668715e+13,2.140719e+13,1.996177e+13
有利子負債合計,3.879288e+13,3.656178e+13,2.938027e+13,2.649636e+13
...,...,...,...,...
現金及び現金同等物、短期投資,1.591816e+13,1.411423e+13,9.232641e+12,8.620903e+12
その他の短期投資,6.935759e+12,4.702168e+12,1.715675e+12,2.507248e+12
現金及び現金同等物,8.982404e+12,9.412060e+12,7.516966e+12,6.113655e+12
現金同等物,2.637713e+12,3.166803e+12,1.568669e+12,1.482773e+12


In [ ]:
# 項目名のマッピング (損益計算書に関連する項目を追加)
japanese_names.update({
    'Tax Effect Of Unusual Items': '特別損益に係る税効果',
    'Tax Rate For Calcs': '計算用税率',
    'Normalized EBITDA': '調整後EBITDA',
    'Total Unusual Items': '特別損益合計',
    'Total Unusual Items Excluding Goodwill': 'のれんを除く特別損益合計',
    'Net Income From Continuing Operation Net Minority Interest': '継続事業からの純利益 (非支配持分控除後)',
    'Reconciled Depreciation': '調整後減価償却費',
    'Reconciled Cost Of Revenue': '調整後売上原価',
    'EBITDA': 'EBITDA',
    'EBIT': 'EBIT',
    'Net Interest Income': '純受取利息',
    'Interest Expense': '支払利息',
    'Interest Income': '受取利息',
    'Normalized Income': '調整後純利益',
    'Net Income From Continuing And Discontinued Operation': '継続・非継続事業からの純利益',
    'Total Expenses': '費用合計',
    'Total Operating Income As Reported': '報告済み営業利益合計',
    'Diluted Average Shares': '希薄化後平均株式数',
    'Basic Average Shares': '基本平均株式数',
    'Diluted EPS': '希薄化後EPS',
    'Basic EPS': '基本EPS',
    'Diluted NI Availto Com Stockholders': '希薄化後親会社株主に帰属する当期純利益',
    'Average Dilution Earnings': '平均希薄化利益',
    'Net Income Common Stockholders': '親会社株主に帰属する当期純利益',
    'Otherunder Preferred Stock Dividend': '優先株式配当金控除',
    'Net Income': '当期純利益',
    'Minority Interests': '非支配持分',
    'Net Income Including Noncontrolling Interests': '非支配持分を含む純利益',
    'Net Income Continuous Operations': '継続事業からの純利益',
    'Tax Provision': '税金費用',
    'Pretax Income': '税引前利益',
    'Other Income Expense': 'その他の収益費用',
    'Other Non Operating Income Expenses': 'その他の営業外収益費用',
    'Earnings From Equity Interest': '持分法による投資損益',
    'Gain On Sale Of Security': '有価証券売却益',
    'Net Non Operating Interest Income Expense': '純営業外受取利息費用',
    'Total Other Finance Cost': 'その他の金融費用合計',
    'Interest Expense Non Operating': '営業外支払利息',
    'Interest Income Non Operating': '営業外受取利息',
    'Operating Income': '営業利益',
    'Operating Expense': '営業費用',
    'Selling General And Administration': '販売費及び一般管理費',
    'Gross Profit': '売上総利益',
    'Cost Of Revenue': '売上原価',
    'Total Revenue': '売上高合計',
    'Operating Revenue': '営業収益'
})


# income_statement DataFrameのカラム名を日本語にリネーム
# エラーを避けるため、存在しないカラムは無視します
income_statement_japanese = income_statement.rename(index=japanese_names)

# 日本語カラム名でDataFrameを表示
print("--- 損益計算書 (日本語項目名) ---")
display(income_statement_japanese)

--- 損益計算書 (日本語項目名) ---


,2025-03-31,2024-03-31,2023-03-31,2022-03-31,2021-03-31
特別損益に係る税効果,2.089089e+11,8.561091e+10,7.482368e+10,8.708560e+10,NaN
計算用税率,2.530000e-01,2.720000e-01,3.200000e-01,2.800000e-01,NaN
調整後EBITDA,7.924202e+12,8.802138e+12,5.522169e+12,5.533850e+12,NaN
特別損益合計,8.257270e+11,3.147460e+11,2.338240e+11,3.110200e+11,NaN
のれんを除く特別損益合計,8.257270e+11,3.147460e+11,2.338240e+11,3.110200e+11,NaN
継続事業からの純利益 (非支配持分控除後),4.765086e+12,4.944933e+12,2.451318e+12,2.850110e+12,NaN
調整後減価償却費,2.251233e+12,2.087066e+12,2.039904e+12,1.821880e+12,NaN
調整後売上原価,3.845867e+13,3.572701e+13,3.084128e+13,2.540783e+13,NaN
EBITDA,8.749929e+12,9.116884e+12,5.755993e+12,5.844870e+12,NaN
EBIT,6.498696e+12,7.029818e+12,3.716089e+12,4.022990e+12,NaN


In [ ]:
japanese_names.update({
    'Free Cash Flow': 'フリーキャッシュフロー',
    'Repayment Of Debt': '借入金の返済',
    'Issuance Of Debt': '借入金による調達',
    'Capital Expenditure': '設備投資',
    'End Cash Position': '期末現金残高',
    'Beginning Cash Position': '期首現金残高',
    'Effect Of Exchange Rate Changes': '為替変動の影響',
    'Changes In Cash': '現金増減額',
    'Financing Cash Flow': '財務活動によるキャッシュフロー',
    'Cash Flow From Continuing Financing Activities': '継続事業に係る財務活動によるキャッシュフロー',
    'Net Other Financing Charges': 'その他の財務費用純額',
    'Cash Dividends Paid': '配当金の支払額',
    'Common Stock Dividend Paid': '普通株式配当金の支払額',
    'Net Common Stock Issuance': '普通株式の発行による収入純額',
    'Net Issuance Payments Of Debt': '借入金による調達・返済純額',
    'Net Short Term Debt Issuance': '短期借入金による調達純額',
    'Net Long Term Debt Issuance': '長期借入金による調達純額',
    'Long Term Debt Payments': '長期借入金の返済額',
    'Long Term Debt Issuance': '長期借入金による調達額',
    'Investing Cash Flow': '投資活動によるキャッシュフロー',
    'Cash Flow From Continuing Investing Activities': '継続事業に係る投資活動によるキャッシュフロー',
    'Net Other Investing Changes': 'その他の投資変動純額',
    'Net Investment Purchase And Sale': '投資有価証券の取得及び売却純額',
    'Sale Of Investment': '投資有価証券の売却による収入',
    'Purchase Of Investment': '投資有価証券の取得による支出',
    'Net Intangibles Purchase And Sale': '無形固定資産の取得及び売却純額',
    'Purchase Of Intangibles': '無形固定資産の取得による支出',
    'Net PPE Purchase And Sale': '有形固定資産の取得及び売却純額',
    'Sale Of PPE': '有形固定資産の売却による収入',
    'Purchase Of PPE': '有形固定資産の取得による支出',
    'Operating Cash Flow': '営業活動によるキャッシュフロー',
    'Cash Flow From Continuing Operating Activities': '継続事業に係る営業活動によるキャッシュフロー',
    'Taxes Refund Paid': '税金還付・支払額',
    'Interest Received Cfo': '営業活動による受取利息',
    'Interest Paid Cfo': '営業活動による支払利息',
    'Dividend Received Cfo': '営業活動による受取配当金',
    'Change In Working Capital': '運転資本の増減額',
    'Change In Other Working Capital': 'その他の運転資本の増減額',
    'Change In Other Current Liabilities': 'その他の流動負債の増減額',
    'Change In Other Current Assets': 'その他の流動資産の増減額',
    'Change In Payables And Accrued Expense': '買入債務及び未払費用の増減額',
    'Change In Payable': '買入債務の増減額',
    'Change In Inventory': '棚卸資産の増減額',
    'Change In Receivables': '売上債権の増減額',
    'Changes In Account Receivables': '売掛金の増減額',
    'Other Non Cash Items': 'その他の非資金項目',
    'Deferred Tax': '繰延税金',
    'Deferred Income Tax': '繰延税金費用',
    'Depreciation Amortization Depletion': '減価償却費及び償却費',
    'Depreciation And Amortization': '減価償却費及び償却費',
    'Operating Gains Losses': '営業損益',
    'Earnings Losses From Equity Investments': '持分法による投資損益'
})

# cashflow_statement DataFrameのカラム名を日本語にリネーム
# エラーを避けるため、存在しないカラムは無視します
cashflow_statement_japanese = cashflow_statement.rename(index=japanese_names)

# 日本語カラム名でDataFrameを表示
print("\n--- キャッシュフロー計算書 (日本語項目名) ---")
display(cashflow_statement_japanese)


--- キャッシュフロー計算書 (日本語項目名) ---


,2025-03-31,2024-03-31,2023-03-31,2022-03-31
フリーキャッシュフロー,-1.560993e+12,-8.420210e+11,-7.507560e+11,-1.076290e+11
借入金の返済,-1.087226e+13,-8.752329e+12,-8.353033e+12,-8.843665e+12
借入金による調達,1.338158e+13,1.205735e+13,9.276918e+12,8.122678e+12
設備投資,-5.257927e+12,-5.048394e+12,-3.705832e+12,-3.830244e+12
期末現金残高,8.982404e+12,9.412060e+12,7.516966e+12,6.113655e+12
期首現金残高,9.412060e+12,7.516966e+12,6.113655e+12,5.100857e+12
為替変動の影響,-1.340890e+11,1.899140e+11,1.033050e+11,3.341950e+11
現金増減額,-2.955670e+11,1.705180e+12,1.300006e+12,6.786030e+11
財務活動によるキャッシュフロー,1.972360e+11,2.497558e+12,-5.618000e+10,-2.466516e+12
継続事業に係る財務活動によるキャッシュフロー,1.972350e+11,2.497558e+12,-5.618100e+10,-2.466516e+12
